### In this program, I firstly use the large aligned results to locate subset, then match within the subset

In [8]:
import pickle as pkl
import json
from collections import defaultdict

In [13]:
"""
Organize index within one episode
Input: ([[]])
Output: {index: []}
"""
def get_index_dict(episode):
    index_dict = defaultdict()
    for idx, segment, en_sub, zh_sub in episode:
        temp = [segment, en_sub, zh_sub]
        if idx not in index_dict:
            index_dict[idx] = [temp]
        else:
            index_dict[idx].append(temp)
    return index_dict

In [51]:
# Organize data
def organize_by_seasons(all_data):
    res = {}
    for epi in list(all_data.keys()):
        season = int(epi[1:3])
        episode = int(epi[-2:])
        # Process the data in one episode
        temp = get_index_dict(all_data[epi])
        if season not in res:
            res[season] = {
                episode: temp
            }
        else:
            res[season][episode] = temp
    return res

In [68]:
"""
Get all the indexs within one episode
Input: {index: [[]]}
Output: sorted index list
"""
def get_epi_indexs_gaps(episode):
    idx_list = []
    gaps = []
    for idx in episode:
        idx_list.append(idx)
    idx_list.sort()
    for i in range(len(idx_list)-1):
        gaps.append(idx_list[i+1]-idx_list[i])
    return idx_list, gaps

In [81]:
"""
Locate continuous subset that gap between to indexs is small than threshold
Input: indexs, gaps, threshold
Output: indexs of continuous subset
"""
def find_all_continuous_subsets(idx_list, gaps, len_threshold, gap_threshold):
    res = []
    path = [idx_list[0]]
    for i in range(len(gaps)):
        if gaps[i]<=gap_threshold:
            path.append(idx_list[i+1])
        else:
            if len(path)>=len_threshold:
                res.append(path)
            path = [idx_list[i+1]]
    return res

In [56]:
# Load search result
with open('episode_indexs_transformed.pkl', 'rb') as f:
    temp = pkl.load(f)
results = organize_by_seasons(temp)

In [84]:
temp = results[4][10]
idx_list, gaps = get_epi_indexs_gaps(temp)
print(len(idx_list), len(gaps))
subsets = find_all_continuous_subsets(idx_list, gaps, 6, 100)
for x in subsets:
    print(x)
print(len(subsets[0]))

148 147
[5896112, 5896113, 5896164, 5896166, 5896167, 5896211, 5896212, 5896213, 5896214, 5896222, 5896223, 5896224, 5896226, 5896227, 5896237, 5896252, 5896253, 5896258, 5896280, 5896281, 5896282, 5896289, 5896319, 5896355, 5896357, 5896358, 5896360, 5896361, 5896377, 5896378, 5896379, 5896380, 5896384]
33


In [80]:
print(gaps)

[56, 34409, 32093, 15075, 15366, 29271, 12295, 109286, 48306, 37169, 175114, 206422, 18252, 61731, 223584, 115973, 12073, 34691, 75659, 122384, 12973, 125641, 87172, 80949, 248022, 36631, 104, 2, 94397, 152948, 52439, 126241, 52631, 58950, 116236, 138090, 72149, 6603, 24994, 30238, 31854, 32318, 146106, 6819, 18601, 164302, 249708, 32563, 249541, 52701, 3552, 52996, 169839, 31107, 9349, 71447, 171130, 46412, 21805, 76797, 39510, 151898, 50685, 14807, 29907, 199072, 45144, 75315, 306718, 8388, 18370, 71940, 7459, 153055, 30943, 7132, 126040, 1, 51, 2, 1, 44, 1, 1, 1, 8, 1, 1, 2, 1, 10, 15, 1, 5, 22, 1, 1, 7, 30, 36, 2, 1, 2, 1, 16, 1, 1, 1, 4, 227073, 32627, 17289, 1513, 50989, 101794, 371561, 151946, 86017, 229713, 87655, 498317, 177261, 234792, 31342, 14639, 78926, 45713, 35098, 169682, 35087, 18622, 14280, 21550, 55827, 69547, 55711, 44843, 36177, 63815, 19984, 24118, 70974, 153327, 332805, 297432, 69994, 26838]


In [70]:
print(idx_list)

[24163, 24219, 58628, 90721, 105796, 121162, 150433, 162728, 272014, 320320, 357489, 532603, 739025, 757277, 819008, 1042592, 1158565, 1170638, 1205329, 1280988, 1403372, 1416345, 1541986, 1629158, 1710107, 1958129, 1994760, 1994864, 1994866, 2089263, 2242211, 2294650, 2420891, 2473522, 2532472, 2648708, 2786798, 2858947, 2865550, 2890544, 2920782, 2952636, 2984954, 3131060, 3137879, 3156480, 3320782, 3570490, 3603053, 3852594, 3905295, 3908847, 3961843, 4131682, 4162789, 4172138, 4243585, 4414715, 4461127, 4482932, 4559729, 4599239, 4751137, 4801822, 4816629, 4846536, 5045608, 5090752, 5166067, 5472785, 5481173, 5499543, 5571483, 5578942, 5731997, 5762940, 5770072, 5896112, 5896113, 5896164, 5896166, 5896167, 5896211, 5896212, 5896213, 5896214, 5896222, 5896223, 5896224, 5896226, 5896227, 5896237, 5896252, 5896253, 5896258, 5896280, 5896281, 5896282, 5896289, 5896319, 5896355, 5896357, 5896358, 5896360, 5896361, 5896377, 5896378, 5896379, 5896380, 5896384, 6123457, 6156084, 6173373, 6

In [71]:
print(gaps)

[56, 34409, 32093, 15075, 15366, 29271, 12295, 109286, 48306, 37169, 175114, 206422, 18252, 61731, 223584, 115973, 12073, 34691, 75659, 122384, 12973, 125641, 87172, 80949, 248022, 36631, 104, 2, 94397, 152948, 52439, 126241, 52631, 58950, 116236, 138090, 72149, 6603, 24994, 30238, 31854, 32318, 146106, 6819, 18601, 164302, 249708, 32563, 249541, 52701, 3552, 52996, 169839, 31107, 9349, 71447, 171130, 46412, 21805, 76797, 39510, 151898, 50685, 14807, 29907, 199072, 45144, 75315, 306718, 8388, 18370, 71940, 7459, 153055, 30943, 7132, 126040, 1, 51, 2, 1, 44, 1, 1, 1, 8, 1, 1, 2, 1, 10, 15, 1, 5, 22, 1, 1, 7, 30, 36, 2, 1, 2, 1, 16, 1, 1, 1, 4, 227073, 32627, 17289, 1513, 50989, 101794, 371561, 151946, 86017, 229713, 87655, 498317, 177261, 234792, 31342, 14639, 78926, 45713, 35098, 169682, 35087, 18622, 14280, 21550, 55827, 69547, 55711, 44843, 36177, 63815, 19984, 24118, 70974, 153327, 332805, 297432, 69994, 26838]


In [59]:
for season in results:
    for episode in results[season]:
        print("Season:", season, "Episode:", episode)
        temp = results[season][episode]
        print(temp)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [50]:
for season in temp:
    for episode in temp[season]:
        print("Season:", season, "Episode:", episode)

Season: 4 Episode: 17
Season: 4 Episode: 21
Season: 4 Episode: 1
Season: 4 Episode: 24
Season: 4 Episode: 2
Season: 4 Episode: 15
Season: 4 Episode: 19
Season: 4 Episode: 12
Season: 4 Episode: 20
Season: 4 Episode: 16
Season: 4 Episode: 5
Season: 4 Episode: 11
Season: 4 Episode: 7
Season: 4 Episode: 13
Season: 4 Episode: 14
Season: 4 Episode: 18
Season: 4 Episode: 22
Season: 4 Episode: 9
Season: 4 Episode: 3
Season: 4 Episode: 10
Season: 4 Episode: 8
Season: 4 Episode: 23
Season: 4 Episode: 6
Season: 4 Episode: 4
Season: 3 Episode: 20
Season: 3 Episode: 12
Season: 3 Episode: 6
Season: 3 Episode: 22
Season: 3 Episode: 7
Season: 3 Episode: 15
Season: 3 Episode: 9
Season: 3 Episode: 23
Season: 3 Episode: 18
Season: 3 Episode: 8
Season: 3 Episode: 5
Season: 3 Episode: 4
Season: 3 Episode: 17
Season: 3 Episode: 2
Season: 3 Episode: 14
Season: 3 Episode: 10
Season: 3 Episode: 1
Season: 3 Episode: 11
Season: 3 Episode: 16
Season: 3 Episode: 3
Season: 3 Episode: 13
Season: 3 Episode: 19
Season

In [6]:
# Show some examples
epi_keys = list(results.keys())
for x in epi_keys[:2]:
    temp = results[x]
    print(x, len(temp))
    for item in temp:
        print(item)
    print('=='*50)

S04E17 133
[6275973, 'well no sir and do you', 'well no sir and do you know why', '當然不 你知道為什麼嗎?']
[6275975, 'every thursday night for the last', 'every thursday night for the last eight years you and i and our friends have gathered here in this very spot to break bread covered with cheese and sauce discuss the issues of the day and yes share a laugh or two', '過去八年裡每個週四晚上 你 我 還有我們的朋友們 坐在這裡 切開塗滿芝士和香腸的麵餅 談論一下生活 偶爾相視一笑']
[6275975, 'eight years you and i and', 'every thursday night for the last eight years you and i and our friends have gathered here in this very spot to break bread covered with cheese and sauce discuss the issues of the day and yes share a laugh or two', '過去八年裡每個週四晚上 你 我 還有我們的朋友們 坐在這裡 切開塗滿芝士和香腸的麵餅 談論一下生活 偶爾相視一笑']
[6275975, 'our friends have gathered here in', 'every thursday night for the last eight years you and i and our friends have gathered here in this very spot to break bread covered with cheese and sauce discuss the issues of the day and yes share a laugh or two', '